**RNN Tabanlı Şair Yazılımı**

RNN Yılmaz Değirmenci tarafından yazılmış şiirler ile eğitilmektedir.

Orijinal Çalışma: https://karpathy.github.io/2015/05/21/rnn-effectiveness/

Yararlanılan Kaynak: https://github.com/ageron/handson-ml2

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/ageron/handson-ml2/blob/master/16_nlp_with_rnns_and_attention.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

# Kurulum

Ön hazırlık kurulumu. Python 3.5 veya sonrası, ayrıca Scikit-Learn ≥0.20 ve TensorFlow ≥2.0.

In [4]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !pip install -q -U tensorflow-addons
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)


     |████████████████████████████████| 1.1MB 2.8MB/s 


# Char-RNN

## Karakter dizisini karılmış bir yığın penceresine dönüştürme 

Örneğin 0 ile 14 arasındaki değerleri 5'lik yığın pencerelerine bölümleme, her biri iki basamak kayacak şekilde: `[0, 1, 2, 3, 4]`, `[2, 3, 4, 5, 6]`, gibi.), ardından bunları rasgele kararak girdilere bölümle: örneğin `[2, 3, 4, 5, 6]` dizisi `[[2, 3, 4, 5], [3, 4, 5, 6]]`), haline dönüşecektir. Sonra ise 3 öğeden oluşan girdi/hedef yığınları oluştur:

In [41]:
np.random.seed(42)
tf.random.set_seed(42)

n_steps = 5
dataset = tf.data.Dataset.from_tensor_slices(tf.range(15))
dataset = dataset.window(n_steps, shift=2, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(n_steps))
dataset = dataset.shuffle(10).map(lambda window: (window[:-1], window[1:]))
dataset = dataset.batch(3).prefetch(1)
for index, (X_batch, Y_batch) in enumerate(dataset):
    print("_" * 20, "Batch", index, "\nX_batch")
    print(X_batch.numpy())
    print("=" * 5, "\nY_batch")
    print(Y_batch.numpy())

____________________ Batch 0 
X_batch
[[6 7 8 9]
 [2 3 4 5]
 [4 5 6 7]]
===== 
Y_batch
[[ 7  8  9 10]
 [ 3  4  5  6]
 [ 5  6  7  8]]
____________________ Batch 1 
X_batch
[[ 0  1  2  3]
 [ 8  9 10 11]
 [10 11 12 13]]
===== 
Y_batch
[[ 1  2  3  4]
 [ 9 10 11 12]
 [11 12 13 14]]


## Verisetini Yükleme ve Hazırlama

In [0]:
yilmazdegirmenci_text = open("yilmaz-degirmenci-siirler.txt").read().lower()

In [43]:
print(yilmazdegirmenci_text[:148])

﻿yanki

bazen ayazda buz tutmuş ruhum
senin ışığını yansıtan aynalarla ısınıyor,
buzun mercekleşip vücudu yakması gibi bu
yüreğimi derinden derine k


In [44]:
"".join(sorted(set(yilmazdegirmenci_text.lower())))

'\n !",-.19:;?abcdefghijklmnoprstuvyzçöûüğış̇\u200e…\ufeff'

In [0]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(yilmazdegirmenci_text)

In [46]:
tokenizer.texts_to_sequences(["Test!"])

[[17, 3, 13, 17, 42]]

In [47]:
tokenizer.sequences_to_texts([[17, 3, 13, 17, 42]])

['t e s t !']

In [0]:
max_id = len(tokenizer.word_index) # number of distinct characters
dataset_size = tokenizer.document_count # total number of characters

In [0]:
[encoded] = np.array(tokenizer.texts_to_sequences([yilmazdegirmenci_text])) - 1
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [0]:
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

In [0]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [0]:
np.random.seed(42)
tf.random.set_seed(42)

In [0]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [0]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [0]:
dataset = dataset.prefetch(1)

In [56]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 46) (32, 100)


## Modeli Yarat ve Eğit

In [57]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")


In [0]:
history = model.fit(dataset, steps_per_epoch=train_size // batch_size,
                    epochs=10)

model.save('yilmazcan.h5') 

## Modeli Kullanarak Metin Üret

In [0]:
model.load_weights("yilmazcan.h5")

In [0]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [60]:
X_new = preprocess(["guru"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

'r'

In [61]:
tf.random.set_seed(42)

tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=40).numpy()

array([[0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
        2, 0, 0, 1, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 0, 0, 0, 0]])

In [0]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [64]:
tf.random.set_seed(33)

next_char("beni", temperature=1)


'm'

In [65]:
tf.random.set_seed(33)

next_char("guru", temperature=1)

'r'

In [0]:
def complete_text(text, n_chars=100, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [67]:
tf.random.set_seed(39)

print(complete_text("a", temperature=1))

ar
.
 
sefbeni sokak yüküne 
nazzin sonyatı yaşayan sabah uğruna
bırak denmeden renklerler ağaçlarıma


In [68]:
tf.random.set_seed(41)
print(complete_text("b", temperature=0.4))

baktım seni çocuk kalan
bir gün onun gözlerinde doğan gelen
bir bir çağlayan zaman geçerken

bir de b


In [69]:
tf.random.set_seed(44)
print(complete_text("a", temperature=0.5))

aklarını suyun
susuz sezgilerimi
bir halde değil mi?
seni sevmek lazım kanadım

başka bir denizden
gö
